# Grundlagen

## Fallstudie RKI Corona Daten

### Daten einlesen

In [1]:
import pandas as pd

link="https://opendata.arcgis.com/datasets/dd4580c810204019a7b8eb3e0b329dd6_0.csv"
RKI_komplett = pd.read_csv(link)

In [6]:
RKI_komplett.tail()

,ObjectId,IdBundesland,Bundesland,Landkreis,Altersgruppe,Geschlecht,AnzahlFall,AnzahlTodesfall,Meldedatum,IdLandkreis,Datenstand,NeuerFall,NeuerTodesfall,Refdatum,NeuGenesen,AnzahlGenesen,IstErkrankungsbeginn,Altersgruppe2
1786799,1786800,16,Thüringen,LK Altenburger Land,A80+,W,5,0,2021/04/21 00:00:00+00,16077,"28.04.2021, 00:00 Uhr",0,-9,2021/04/21 00:00:00+00,-9,0,0,Nicht übermittelt
1786800,1786801,16,Thüringen,LK Altenburger Land,A80+,W,2,0,2021/04/22 00:00:00+00,16077,"28.04.2021, 00:00 Uhr",0,-9,2021/04/22 00:00:00+00,-9,0,0,Nicht übermittelt
1786801,1786802,16,Thüringen,LK Altenburger Land,A80+,W,1,0,2021/04/23 00:00:00+00,16077,"28.04.2021, 00:00 Uhr",0,-9,2021/04/23 00:00:00+00,-9,0,0,Nicht übermittelt
1786802,1786803,16,Thüringen,LK Altenburger Land,A80+,W,1,0,2021/04/26 00:00:00+00,16077,"28.04.2021, 00:00 Uhr",0,-9,2021/04/26 00:00:00+00,-9,0,0,Nicht übermittelt
1786803,1786804,16,Thüringen,LK Altenburger Land,unbekannt,W,1,0,2021/04/06 00:00:00+00,16077,"28.04.2021, 00:00 Uhr",0,-9,2021/04/06 00:00:00+00,-9,0,0,Nicht übermittelt


### Daten bearbeiten (Fälle für Gesamtdeutschland bestimmen)

In [3]:
RKI_Deutschland = RKI_komplett.groupby(['Meldedatum']).agg({'AnzahlFall': ['sum']}).reset_index()
RKI_Deutschland['Meldedatum'] = pd.to_datetime(RKI_Deutschland['Meldedatum'],format='%Y/%m/%d')
RKI_Deutschland.columns = ['Meldedatum','Neue Fallzahlen']
RKI_Deutschland["Neue Fallzahlen"] = RKI_Deutschland["Neue Fallzahlen"].astype(int)
RKI_Deutschland["Neue Fallzahlen Mittelwert (7 Tage)"] = RKI_Deutschland["Neue Fallzahlen"].rolling(7).mean().fillna(0)
RKI_Deutschland["Neue Fallzahlen Mittelwert (7 Tage)"] = RKI_Deutschland["Neue Fallzahlen Mittelwert (7 Tage)"].astype(int)
RKI_Deutschland["Faelle gesamt"] = RKI_Deutschland["Neue Fallzahlen"].cumsum(axis = 0) 
RKI_Deutschland.to_csv("RKI_Corona_Deutschland.csv", index=False)

In [5]:
RKI_Deutschland.tail()

,Meldedatum,Neue Fallzahlen,Neue Fallzahlen Mittelwert (7 Tage),Faelle gesamt
441,2021-04-23 00:00:00+00:00,22873,20670,3276582
442,2021-04-24 00:00:00+00:00,17820,20687,3294402
443,2021-04-25 00:00:00+00:00,8597,20652,3302999
444,2021-04-26 00:00:00+00:00,11309,20505,3314308
445,2021-04-27 00:00:00+00:00,17841,19051,3332149


### Bundesland rausfiltern

In [7]:
RKI_Bundesland = RKI_komplett.groupby(['Meldedatum','Bundesland']).agg({'AnzahlFall': ['sum']}).reset_index()
RKI_Bundesland.columns=["Meldedatum","Bundesland","Neue Fallzahlen"]
RKI_Bundesland['Meldedatum'] = pd.to_datetime(RKI_Bundesland['Meldedatum'],format='%Y/%m/%d')
RKI_Bundesland.to_csv("RKI_Corona_Bundeslaender.csv", index=False)
RKI_Bundesland

,Meldedatum,Bundesland,Neue Fallzahlen
0,2020-01-07 00:00:00+00:00,Nordrhein-Westfalen,1
1,2020-01-19 00:00:00+00:00,Berlin,1
2,2020-01-23 00:00:00+00:00,Saarland,1
3,2020-01-25 00:00:00+00:00,Nordrhein-Westfalen,1
4,2020-01-28 00:00:00+00:00,Bayern,2
...,...,...,...
6634,2021-04-27 00:00:00+00:00,Saarland,189
6635,2021-04-27 00:00:00+00:00,Sachsen,1022
6636,2021-04-27 00:00:00+00:00,Sachsen-Anhalt,479
6637,2021-04-27 00:00:00+00:00,Schleswig-Holstein,352


### Leipzig rausfiltern

In [8]:
Leipzig = RKI_komplett.loc[RKI_komplett["Landkreis"] == "SK Leipzig"]
Leipzig = Leipzig.groupby(['Meldedatum']).agg({'AnzahlFall': ['sum']}).reset_index()
Leipzig['Meldedatum'] = pd.to_datetime(Leipzig['Meldedatum'],format='%Y/%m/%d')
Leipzig.columns = ['Meldedatum','Neue Fallzahlen']
Leipzig["Neue Fallzahlen Mittelwert (7 Tage)"] = Leipzig["Neue Fallzahlen"].rolling(7).mean().fillna(0).astype(int)
Leipzig["Neue Fallzahlen Summe (7 Tage)"] = Leipzig["Neue Fallzahlen"].rolling(7).sum().fillna(0).astype(int)
Leipzig["Faelle gesamt"] = Leipzig["Neue Fallzahlen"].cumsum(axis = 0) 
Einwohner = 605407  # Quelle: https://www.leipzig.de/news/news/leipzig-hat-jetzt-605407-einwohner/
Leipzig["7 Tage Indizdenz"] = Leipzig["Neue Fallzahlen Summe (7 Tage)"]/Einwohner*100000
Leipzig["7 Tage Indizdenz"] = Leipzig["7 Tage Indizdenz"].round(0)
Leipzig.to_csv("RKI_Corona_Leipzig.csv", index=False)
Leipzig

,Meldedatum,Neue Fallzahlen,Neue Fallzahlen Mittelwert (7 Tage),Neue Fallzahlen Summe (7 Tage),Faelle gesamt,7 Tage Indizdenz
0,2020-03-06 00:00:00+00:00,1,0,0,1,0.0
1,2020-03-08 00:00:00+00:00,1,0,0,2,0.0
2,2020-03-10 00:00:00+00:00,2,0,0,4,0.0
3,2020-03-11 00:00:00+00:00,2,0,0,6,0.0
4,2020-03-12 00:00:00+00:00,10,0,0,16,0.0
...,...,...,...,...,...,...
338,2021-04-23 00:00:00+00:00,86,102,714,19974,118.0
339,2021-04-24 00:00:00+00:00,91,105,741,20065,122.0
340,2021-04-25 00:00:00+00:00,57,107,751,20122,124.0
341,2021-04-26 00:00:00+00:00,38,105,740,20160,122.0
